# Seq2Seq Q&A Chatbot 구현 실습

https://raw.githubusercontent.com/songys/Chatbot_data/refs/heads/master/ChatbotData.csv

In [4]:
import numpy as numpy
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/songys/Chatbot_data/refs/heads/master/ChatbotData.csv')
df = df[['Q', 'A']]
df

Q                         A
0                       12시 땡!                하루가 또 가네요.
1                  1지망 학교 떨어졌어                 위로해 드립니다.
2                 3박4일 놀러가고 싶다               여행은 언제나 좋죠.
3              3박4일 정도 놀러가고 싶다               여행은 언제나 좋죠.
4                      PPL 심하네                눈살이 찌푸려지죠.
...                        ...                       ...
11818           훔쳐보는 것도 눈치 보임.        티가 나니까 눈치가 보이는 거죠!
11819           훔쳐보는 것도 눈치 보임.             훔쳐보는 거 티나나봐요.
11820              흑기사 해주는 짝남.                    설렜겠어요.
11821  힘든 연애 좋은 연애라는게 무슨 차이일까?  잘 헤어질 수 있는 사이 여부인 거 같아요.
11822               힘들어서 결혼할까봐        도피성 결혼은 하지 않길 바라요.

[11823 rows x 2 columns]

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11823 entries, 0 to 11822
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Q       11823 non-null  object
 1   A       11823 non-null  object
dtypes: object(2)
memory usage: 184.9+ KB


### 1. 데이터 전처리

##### 1-1. 토커나이저 학습 (sentencepiece 활용)

- 접두사, 접미사 (bos, eos)

In [9]:

# BOS, EOS 토큰 추가
df['Q'] = df['Q'].apply(lambda x: f"<BOS> {x} <EOS>")
df['A'] = df['A'].apply(lambda x: f"<BOS> {x} <EOS>")

##### 1-2. 학습용 데이터 Q_input, A_input, A_target 생성
- 시퀀싱 처리
- 패딩 처리

In [11]:

from sklearn.model_selection import train_test_split
from transformers import T5Tokenizer

train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)


# T5 토크나이저 로드
tokenizer = T5Tokenizer.from_pretrained("t5-small")

# 추가 특수 토큰 등록 (선택)
special_tokens_dict = {'additional_special_tokens': ['<BOS>', '<EOS>']}
tokenizer.add_special_tokens(special_tokens_dict)

# 최대 토큰 길이 설정
MAX_LEN = 64

def preprocess_data(df, tokenizer, max_len=MAX_LEN):
    Q_input = tokenizer(
        list(df['Q']),
        padding='max_length',
        truncation=True,
        max_length=max_len,
        return_tensors="pt"
    )
    A_input = tokenizer(
        list(df['A']),
        padding='max_length',
        truncation=True,
        max_length=max_len,
        return_tensors="pt"
    )
    # target은 레이블이므로 attention mask 제외
    A_target = A_input['input_ids'].clone()

    return Q_input, A_input, A_target

Q_input, A_input, A_target = preprocess_data(train_df, tokenizer)

print("✅ 시퀀싱 및 패딩 완료")
print("Q_input shape:", Q_input['input_ids'].shape)
print("A_input shape:", A_input['input_ids'].shape)
print("A_target shape:", A_target.shape)



tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

c:\Users\roxie\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\roxie\.cache\huggingface\hub\models--t5-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


✅ 시퀀싱 및 패딩 완료
Q_input shape: torch.Size([10640, 64])
A_input shape: torch.Size([10640, 64])
A_target shape: torch.Size([10640, 64])


### 2. 모델 생성 및 학습

##### 2-1. 인코더 생성

In [15]:
import torch
import torch.nn as nn

class Encoder(nn.Module):
    def __init__(self, vocab_size, emb_dim, hid_dim, n_layers=1, dropout=0.5):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim)
        self.rnn = nn.GRU(
            input_size=emb_dim,
            hidden_size=hid_dim,
            num_layers=n_layers,
            dropout=dropout if n_layers > 1 else 0,
            batch_first=True
        )
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src, src_mask=None):
        # src : [batch_size, seq_len]
        embedded = self.dropout(self.embedding(src))  # [batch_size, seq_len, emb_dim]
        outputs, hidden = self.rnn(embedded)          # outputs: [batch, seq_len, hid_dim], hidden: [n_layers, batch, hid_dim]
        return outputs, hidden
    
# 샘플 입력 데이터 (Q_input에서 추출)
input_ids = Q_input['input_ids'][:4]  # batch 4개만
vocab_size = tokenizer.vocab_size + len(tokenizer.additional_special_tokens)
emb_dim = 256
hid_dim = 512
n_layers = 2

# 인코더 인스턴스 생성
encoder = Encoder(vocab_size, emb_dim, hid_dim, n_layers, dropout=0.3)

# forward test
outputs, hidden = encoder(input_ids)

print("✅ 인코더 작동 완료!")
print("입력 시퀀스 크기:", input_ids.shape)
print("출력 features 크기:", outputs.shape)
print("히든 상태 크기:", hidden.shape)



IndexError: index out of range in self

##### 2-2. 디코더(teacher-forcing 모델) 생성

### 3. 학습

##### 3-1. 모델 추론

### 4. 디코더 (추론 모델) 생성

##### 4-1. 추론 함수 생성

### 5. 테스트

### 6. 간단한 Chatbot 구현
- 사용자의 입력을 받아 (처리)
- 추론 함수에 전달해서
3. 응답을 추루력
4. 1~3 '종료' 전까지 반복